In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['fake-review-dm', 'final-mergedtsv']


In [2]:
latest =pd.read_csv("../input/final-mergedtsv/final_merged.tsv", header = 0,sep="\t")

In [3]:
latest.head()

,User_Id,Prod_Id,Date_x,Review,Rating,Label,Product_Name,Avg_Prod_Rating,Avg_user_rating,Review_Len,user_total_reviews,corpus,compound,neg,neu,pos,number_Cap_Words,number_digit_Words,noun_count
0,923,0,08-12-2014,The food at snack is a selection of popular Gr...,3,-1,Snack,4.009524,4.435897,215,39,food snack select popular greek dish appet tra...,0.6486,0.061,0.693,0.247,0,0,3
1,923,19,14-01-2014,The restaurant is on the ground floor of a typ...,5,-1,Palo Santo,4.037152,4.435897,513,39,littl place soho wonder lamb sandwich glass wi...,-0.1280,0.189,0.673,0.138,0,0,9
2,923,40,30-05-2014,Really nice mousaka and lovely décor inside. A...,4,-1,Pylos,4.312869,4.435897,231,39,order lunch snack last friday time noth miss f...,0.7717,0.067,0.628,0.305,0,1,4
3,923,63,13-11-2014,I really enjoyed brunch at Jane. The ambiance ...,4,-1,Jane,3.937181,4.435897,169,39,beauti quaint littl restaur pretti street stro...,0.8910,0.042,0.712,0.246,0,0,2
4,923,79,30-03-2014,We ate at the Blue Ribbon with colleagues. The...,3,-1,Blue Ribbon Brasserie,4.280000,4.435897,255,39,snack great place casual sit lunch especi cold...,0.9769,0.000,0.661,0.339,0,0,3


In [4]:
latest.dropna(axis=0, inplace=True)

**Split data for Fake Reviews & True Reviews to collect common words betweeen both to use as stop words**

In [5]:
positive = latest[latest['Label']== 1]
fake = latest[latest['Label'] == -1]
true_word_list = []

In [6]:
for i in range(0,positive['corpus'].count()):
   true_word_temp = positive['corpus'].iloc[i].split()
   true_word_list.append(true_word_temp)
from itertools import chain
list1 = list(chain.from_iterable(true_word_list))
from collections import Counter 
true_mc = Counter(list1).most_common()
 
 

fake_word_list= fake['corpus'].str.split(expand=True).stack()
from collections import Counter 
fake_mc = Counter(fake_word_list).most_common() 

 

**collects top 20 common words -- > stored in df_common**

In [7]:
df_true = pd.DataFrame(true_mc)
df_false = pd.DataFrame(fake_mc)
df_false1 = df_false[0:23]
df_true1 = df_true[0:23]
df_common = list(set(df_true1.iloc[:,0]) & set(df_false1.iloc[:,0]))
df_common

['come',
 'one',
 'tri',
 'best',
 'get',
 'order',
 'love',
 'place',
 'also',
 'servic',
 'time',
 'delici',
 'back',
 'great',
 'like',
 'food',
 'wait',
 'restaur',
 'good',
 'realli',
 'would',
 'go']

In [8]:
latest.head()

,User_Id,Prod_Id,Date_x,Review,Rating,Label,Product_Name,Avg_Prod_Rating,Avg_user_rating,Review_Len,user_total_reviews,corpus,compound,neg,neu,pos,number_Cap_Words,number_digit_Words,noun_count
0,923,0,08-12-2014,The food at snack is a selection of popular Gr...,3,-1,Snack,4.009524,4.435897,215,39,food snack select popular greek dish appet tra...,0.6486,0.061,0.693,0.247,0,0,3
1,923,19,14-01-2014,The restaurant is on the ground floor of a typ...,5,-1,Palo Santo,4.037152,4.435897,513,39,littl place soho wonder lamb sandwich glass wi...,-0.1280,0.189,0.673,0.138,0,0,9
2,923,40,30-05-2014,Really nice mousaka and lovely décor inside. A...,4,-1,Pylos,4.312869,4.435897,231,39,order lunch snack last friday time noth miss f...,0.7717,0.067,0.628,0.305,0,1,4
3,923,63,13-11-2014,I really enjoyed brunch at Jane. The ambiance ...,4,-1,Jane,3.937181,4.435897,169,39,beauti quaint littl restaur pretti street stro...,0.8910,0.042,0.712,0.246,0,0,2
4,923,79,30-03-2014,We ate at the Blue Ribbon with colleagues. The...,3,-1,Blue Ribbon Brasserie,4.280000,4.435897,255,39,snack great place casual sit lunch especi cold...,0.9769,0.000,0.661,0.339,0,0,3


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

**Split original data into train and test **

In [10]:
sentences_train, sentences_test, y_train, y_test = train_test_split(latest.drop('Label',axis=1), latest['Label'], test_size=0.25, random_state=101)

**Using Count Vectorizer only on corpus column of train data set to generate features 
also using ngram = 3 and removing stop words (df_common)**

In [11]:
vectorizer = CountVectorizer(stop_words= df_common, ngram_range=(2,3), max_features= 15000)
vectorizer.fit(sentences_train.corpus)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=15000, min_df=1,
        ngram_range=(2, 3), preprocessor=None,
        stop_words=['come', 'one', 'tri', 'best', 'get', 'order', 'love', 'place', 'also', 'servic', 'time', 'delici', 'back', 'great', 'like', 'food', 'wait', 'restaur', 'good', 'realli', 'would', 'go'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

**Transforming text values to numeric using above vectorizer on train and test dataset**

In [12]:
x_train = vectorizer.transform(sentences_train.corpus)
x_test  = vectorizer.transform(sentences_test.corpus)

**Merging vectorised features with our extracted features using scipy library**

In [13]:
import scipy as sp
final_train = sp.sparse.hstack((x_train,sentences_train[['Rating','Avg_Prod_Rating','Avg_user_rating','Review_Len','user_total_reviews','compound','number_Cap_Words','number_digit_Words','noun_count']].values),format='csr')
final_test = sp.sparse.hstack((x_test,sentences_test[['Rating','Avg_Prod_Rating','Avg_user_rating','Review_Len','user_total_reviews','compound','number_Cap_Words','number_digit_Words','noun_count']].values),format='csr')

**Using SMOTE to handle imbalanced dataset**

In [14]:
from imblearn.over_sampling import SMOTE
ros = SMOTE(random_state=0)
final_train_dataset, y_dataset = ros.fit_resample(final_train, y_train)
#final_train_dataset = final_train
#y_dataset = y_train

Using TensorFlow backend.


**Model to be fit**

In [15]:
#ML Model
classifier = LogisticRegression()
classifier.fit(final_train_dataset, y_dataset)
score = classifier.score(final_test, y_test)

print("Accuracy:", score)


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 0.6458084619898423


**Accuracy + Report + Confusion Matrix**

In [16]:
from sklearn.metrics import confusion_matrix, classification_report
predictions = classifier.predict(final_test)
print(classification_report(y_test, predictions))
confusion_matrix(y_test, predictions)

              precision    recall  f1-score   support

          -1       0.17      0.60      0.26     10398
           1       0.93      0.65      0.77     88444

   micro avg       0.65      0.65      0.65     98842
   macro avg       0.55      0.63      0.51     98842
weighted avg       0.85      0.65      0.71     98842



array([[ 6231,  4167],
       [30842, 57602]])